In [ ]:
import numpy as np
import pandas as pd
import cv2
import time
from skimage.metrics import structural_similarity as ssim
import triplerecovery as tr
from PIL import Image
from os import walk

In [ ]:
openmode, modestr=cv2.IMREAD_COLOR, 'rgb'
dir_path='./images/standard/'
output_path='./psnr_results/'

In [ ]:
files = []
for (dir_path, dir_names, file_names) in walk(dir_path):
    files.extend([dir_path+"/"+f for f in file_names])
len(files)

In [ ]:
files[-5]

In [ ]:
columns = ['file','mode','size','test' ,'psnr_watermarked','ssmi_watermarked','time_watermark', 'psnr_edited','time_authenticate','psnr_recovered','ssmi_recovered','time_recovered']

In [ ]:
df = pd.DataFrame(columns=columns)

In [ ]:
def get_len(l, size):
  return int(l*(size/100))

First tests for the 75%

In [ ]:
# start x start y and end x end y. all are in percentages
tests75=[
  [[0,0,75,100]],# Left zone LZ
  [[0,25,100,100]],# Bottom zone BZ
  [[25,0,100,100]],# Right zone RZ
  [[0,0,100,75]],#f Top zone TZ
  [[26,0,74,100],
   [0,26,100,74]],# Center zone CZ
  [[0,0,100,25],
   [0,0,25,100],
   [75,0,100,100],
   [0,75,100,100]]#  qouter zone OZ
]
tests75_names=['75_LZ','75_BZ','75_RZ','75_TZ','75_CZ','75_OZ']

tests50=[
  [[0,0,100,50]],# Top zone TZ
  [[0,0,50,100]],# Left zone LZ
  [[25,0,75,100]],# Verticle Center zone VCZ
  [[0,25,100,75]],# Horizontal center zone HCZ
  [[0,0,25,100],
   [75,0,100,100]],# left right zone LRZ
  [[0,0,50,50],
   [50,50,100,100]]# up-left down-right zone  ULDRZ
]
tests50_names=['50_TZ','50_LZ','50_VCZ','50_HCZ','50_LRZ','50_ULDRZ']

tests25=[
  [[25,25,75,75]],# center zone CZ
  [[0,0,50,50]],# up-left zone UL
  [[50,25,100,75]],# center right zone CRZ
  [[0,25,50,75]],# center left zone CLZ
  [[25,0,75,50]],# center up zone CUZ
  [[0,25,25,50],
   [75,25,100,50],
   [25,75,75,100]],# different zone 1 DZ1
  [[0,0,25,25],
   [75,0,100,25],
   [0,50,25,75],
   [75,50,100,75]]# different zone 2 DZ2
]
tests25_names=['25_CZ','25_UL','25_CRZ','25_CLZ','25_CUZ','25_DZ1','25_DZ2']

tests=[(tests75,tests75_names),(tests50,tests50_names),(tests25,tests25_names)]

trying on single image

In [ ]:
files

In [ ]:
# files=['./images/standard//mandril_color.tif','./images/standard//cat.jpg','./images/standard//peppers_color.tif','./images/standard//yourname.png', './images/standard//lena_color_512.tif', './images/standard//lena_color_256.tif', './images/standard//art.png']
files=['./images/standard/miku.jpg']

In [ ]:
# f=files[2]
# f

In [ ]:
for f in files:
  print(f)
  imarr=cv2.imread(f, openmode)
  try:
    w= tr.embed(imarr.copy())
    cv2.imwrite(output_path+f.split('//')[-1].split('.')[0]+'_'+modestr+'_watermarked.png',w.imarr)

    for alpha in tests:
      for t in range(len(alpha[0])):
        test=alpha[0][t]
        imarr_watermarked =w.imarr.copy()
        psnr_watermarked=cv2.PSNR(imarr,imarr_watermarked)
        ssmi_watermarked=ssim(imarr,imarr_watermarked,channel_axis=2)
        time_watermark=w.time

        for k in range (len(test)):
          for i in range(get_len(imarr.shape[1],test[k][0]),get_len(imarr.shape[1],test[k][2])):
            for j in range(get_len(imarr.shape[0],test[k][1]),get_len(imarr.shape[0],test[k][3])):
              if imarr_watermarked.ndim>2:
                imarr_watermarked[j,i,:]=0
              else:imarr_watermarked[j,i]=0
              
        psnr_edited=cv2.PSNR(imarr,imarr_watermarked)
        a=tr.authenticate(imarr_watermarked.copy())
        r=tr.recover(imarr_watermarked)
        imarr_recovered=r.imarr
        time_recovered=r.time
        psnr_recovered= cv2.PSNR(imarr,imarr_recovered)
        ssmi_recovered=ssim(imarr,imarr_recovered,channel_axis=2)
        # inseart to the dataframe
        df.loc[len(df)] = [f,modestr,str(imarr.shape),alpha[1][t],psnr_watermarked,ssmi_watermarked,time_watermark,psnr_edited,a.time,psnr_recovered,ssmi_recovered,time_recovered]

        # saving files to there own folder
        # cv2.imwrite(output_path+f.split('//')[-1].split('.')[0]+'_'+alpha[1][t]+'_watermarked.png',w.imarr)
        cv2.imwrite(output_path+f.split('//')[-1].split('.')[0]+'_'+alpha[1][t]+'_'+modestr+'_edited.png',imarr_watermarked)
        cv2.imwrite(output_path+f.split('//')[-1].split('.')[0]+'_'+alpha[1][t]+'_'+modestr+'_mask.png',a.maskarr)
        cv2.imwrite(output_path+f.split('//')[-1].split('.')[0]+'_'+alpha[1][t]+'_'+modestr+'_recovered.png',imarr_recovered)
  except Exception as e:
      print("error",f, imarr.shape)
      print(e)
  df.to_csv(output_path+str(int(time.time()))+'_'+modestr+'psnr_results.csv',index=False)


In [ ]:
df.to_csv(output_path+str(int(time.time()))+'_'+modestr+'psnr_results.csv',index=False)